# TV Script Generation
In this project, we'll generate our own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.I've used part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network that is build later, will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## The Data

In [21]:
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Exploring the Data


In [22]:
view_sentence_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implementing Preprocessing Functions
The first thing to do to any dataset is preprocessing.I've implemented following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
This is used to transform the words to ids.  In this function, I created two dictionaries:
- Dictionary to go from the words to an id: `vocab_to_int`
- Dictionary to go from the id to word: `int_to_vocab`

The following tuple `(vocab_to_int, int_to_vocab)` is used to return these dictionaries.

In [87]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
 
    # TODO: Implement Function
    # this is done to give a smaller number to most frequent word to reduce computation
    """
    counts = Counter(text)
    vocab = sorted(counts,key = counts.get, reverse = True)
    vocab_to_int = {word:ii for ii, word in enumerate(vocab,1)}
    #print(vocab_to_int.items())
    int_to_vocab = {ii:word for word,ii in vocab_to_int.items()}
    #print(int_to_vocab.items())
    return (vocab_to_int, int_to_vocab)
    or
    """
    unique_words = []
    vocab_to_int = {}
    int_to_vocab = {}
    for word in text:
        if word not in unique_words:
            unique_words.append(word)
    for idx, word in enumerate(unique_words, 0):
        vocab_to_int[word] = idx
        int_to_vocab[idx] = word
    return vocab_to_int, int_to_vocab



tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenizing Punctuation
Th script will be splitted into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

I've implemented the function `token_lookup` below to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||". A dictionary is made for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. 

In [88]:
def token_lookup():
 
    # TODO: Implement Function
    return {
        ".": "||PERIOD||",
        ",": "||COMMA||",
        "\"": "||QUOTATION_MARK||",
        ";": "||SEMICOLON||",
        "!": "||EXCLAMATION_MARK||",
        "?": "||QUESTION_MARK||",
        "(": "||LEFT_PARANTHESES||",
        ")": "||RIGHT_PARANTHESES||",
        "--": "||DASH||",
        "\n": "||RETURN||"
    }

tests.test_tokenize(token_lookup)

Tests Passed


## Preprocessing and saving the data
Running the code cell below will preprocess all the data and save it to file.

In [89]:
# Preprocessing Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
The preprocessed data has been saved to disk.

In [90]:
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Building the Neural Network
The components necessary to build a RNN are implemented through the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

In [91]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Checking TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Checking for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


In [92]:
def get_inputs():
   
    input = tf.placeholder(tf.int32, shape=(None,None), name = 'input')
    Targets = tf.placeholder(tf.int32,shape=(None,None), name = 'Targets')
    LearningRate = tf.placeholder(tf.float32, name ='LearningRate')
    return (input,Targets,LearningRate)


tests.test_get_inputs(get_inputs)

Tests Passed


### Building RNN Cell and Initializing


In [93]:
def get_init_cell(batch_size, rnn_size):
          
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm,lstm])
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name='initial_state')
    return (cell,initial_state)


tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Returns the embedded sequence.

In [94]:
def get_embed(input_data, vocab_size, embed_dim):
  
    """
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embeddings = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed_seq = tf.nn.embedding_lookup(embeddings, input_data, name='embed_seq')
    return embed_seq



tests.test_get_embed(get_embed)

Tests Passed


### Building RNN
I've created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Step 1:Building the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
- Step 2:Applying the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)


In [95]:
def build_rnn(cell, inputs):
    """
    Creating a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name='final_state')
    return (outputs, final_state)

tests.test_build_rnn(build_rnn)

Tests Passed


### Building the Neural Network
The functions above are implemented for:
- Applying embedding to `input_data` using the `get_embed(input_data, vocab_size, embed_dim)` function defined above.
- Building RNN using `cell` and the `build_rnn(cell, inputs)` function.
- Applying a fully connected layer with a linear activation and `vocab_size` as the number of outputs.



In [96]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
   
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    return logits, final_state

    



tests.test_build_nn(build_nn)

Tests Passed


### Batches
Defining a function `get_batches` to create batches of input and targets using `int_text`.  The batches are a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [80]:
def get_batches(int_text, batch_size, seq_length):
    """
    Returns batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    
    length = len(int_text)
    num_batches = length // (batch_size * seq_length)
    num_elements = num_batches * batch_size
    x = np.array(int_text)
    y = np.append(int_text[1:length], [x[0]])
    batches = np.empty((num_batches, 2, batch_size, seq_length), dtype=np.int32)
    batch_idx = 0
    seq_idx = 0
    
    for idx in range(0, num_elements):
        start_idx = idx * seq_length
        end_idx = idx * seq_length + seq_length
        batches[batch_idx][0][seq_idx] = x[start_idx:end_idx]
        batches[batch_idx][1][seq_idx] = y[start_idx:end_idx]
        # Corrects the last target of the last batch
        if idx == num_elements - 1:
            batches[batch_idx][1][seq_idx][seq_length - 1] = x[0]
        batch_idx += 1
        #Restarts the cycle w/ the next sequence of the first batch 
        if batch_idx >= num_batches:
            batch_idx = 0
            seq_idx += 1
    return batches
    

tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training- The intuitive part!!
### Hyperparameters
Tuning the following parameters:

- Setting `num_epochs` to the number of epochs.
- Setting `batch_size` to the batch size.
- Setting `rnn_size` to the size of the RNNs.
- Setting `embed_dim` to the size of the embedding.
- Setting `seq_length` to the length of sequence.
- Setting `learning_rate` to the learning rate.
- Setting `show_every_n_batches` to the number of batches the neural network should print progress.

In [84]:
# Number of Epochs
num_epochs = 256
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 200
# Sequence Length
seq_length = 30
# Learning Rate
learning_rate = 0.002
# Shows stats for every n number of batches
show_every_n_batches = 8


save_dir = './save'

### Building the Graph


In [85]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size,embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Training the Neural Net

In [86]:
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Shows every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/17   train_loss = 8.822
Epoch   0 Batch    8/17   train_loss = 6.523
Epoch   0 Batch   16/17   train_loss = 6.384
Epoch   1 Batch    7/17   train_loss = 6.091
Epoch   1 Batch   15/17   train_loss = 6.016
Epoch   2 Batch    6/17   train_loss = 6.026
Epoch   2 Batch   14/17   train_loss = 6.062
Epoch   3 Batch    5/17   train_loss = 5.969
Epoch   3 Batch   13/17   train_loss = 6.043
Epoch   4 Batch    4/17   train_loss = 6.068
Epoch   4 Batch   12/17   train_loss = 5.941
Epoch   5 Batch    3/17   train_loss = 6.027
Epoch   5 Batch   11/17   train_loss = 6.104
Epoch   6 Batch    2/17   train_loss = 5.963
Epoch   6 Batch   10/17   train_loss = 6.046
Epoch   7 Batch    1/17   train_loss = 5.997
Epoch   7 Batch    9/17   train_loss = 5.933
Epoch   8 Batch    0/17   train_loss = 5.946
Epoch   8 Batch    8/17   train_loss = 6.022
Epoch   8 Batch   16/17   train_loss = 5.956
Epoch   9 Batch    7/17   train_loss = 5.998
Epoch   9 Batch   15/17   train_loss = 5.883
Epoch  10 

Epoch  86 Batch    2/17   train_loss = 3.014
Epoch  86 Batch   10/17   train_loss = 3.085
Epoch  87 Batch    1/17   train_loss = 2.985
Epoch  87 Batch    9/17   train_loss = 3.000
Epoch  88 Batch    0/17   train_loss = 2.905
Epoch  88 Batch    8/17   train_loss = 2.980
Epoch  88 Batch   16/17   train_loss = 2.946
Epoch  89 Batch    7/17   train_loss = 2.967
Epoch  89 Batch   15/17   train_loss = 2.855
Epoch  90 Batch    6/17   train_loss = 2.854
Epoch  90 Batch   14/17   train_loss = 2.841
Epoch  91 Batch    5/17   train_loss = 2.781
Epoch  91 Batch   13/17   train_loss = 2.846
Epoch  92 Batch    4/17   train_loss = 2.880
Epoch  92 Batch   12/17   train_loss = 2.835
Epoch  93 Batch    3/17   train_loss = 2.841
Epoch  93 Batch   11/17   train_loss = 2.828
Epoch  94 Batch    2/17   train_loss = 2.775
Epoch  94 Batch   10/17   train_loss = 2.822
Epoch  95 Batch    1/17   train_loss = 2.755
Epoch  95 Batch    9/17   train_loss = 2.745
Epoch  96 Batch    0/17   train_loss = 2.645
Epoch  96 

Epoch 172 Batch    4/17   train_loss = 0.979
Epoch 172 Batch   12/17   train_loss = 1.010
Epoch 173 Batch    3/17   train_loss = 1.037
Epoch 173 Batch   11/17   train_loss = 0.968
Epoch 174 Batch    2/17   train_loss = 1.039
Epoch 174 Batch   10/17   train_loss = 0.978
Epoch 175 Batch    1/17   train_loss = 0.985
Epoch 175 Batch    9/17   train_loss = 0.987
Epoch 176 Batch    0/17   train_loss = 0.956
Epoch 176 Batch    8/17   train_loss = 1.011
Epoch 176 Batch   16/17   train_loss = 0.980
Epoch 177 Batch    7/17   train_loss = 0.992
Epoch 177 Batch   15/17   train_loss = 0.930
Epoch 178 Batch    6/17   train_loss = 0.945
Epoch 178 Batch   14/17   train_loss = 0.978
Epoch 179 Batch    5/17   train_loss = 1.006
Epoch 179 Batch   13/17   train_loss = 0.960
Epoch 180 Batch    4/17   train_loss = 0.927
Epoch 180 Batch   12/17   train_loss = 0.946
Epoch 181 Batch    3/17   train_loss = 0.963
Epoch 181 Batch   11/17   train_loss = 0.969
Epoch 182 Batch    2/17   train_loss = 1.095
Epoch 182 

## Saving Parameters
Saves `seq_length` and `save_dir` for generating a new TV script.

In [97]:
helper.save_params((seq_length, save_dir))

# Checkpoint

In [98]:
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implementing Generate Functions
### Get Tensors
Getting tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name)using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"


In [99]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    return loaded_graph.get_tensor_by_name('input:0'),\
        loaded_graph.get_tensor_by_name('initial_state:0'), \
        loaded_graph.get_tensor_by_name('final_state:0'), \
        loaded_graph.get_tensor_by_name('probs:0')



tests.test_get_tensors(get_tensors)

Tests Passed


### Choosing Word
Implementing the `pick_word()` function to select the next word using `probabilities`.

In [100]:
import random
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    word_int = np.random.choice(len(int_to_vocab.keys()), p=probabilities)
    return int_to_vocab[word_int]
    

tests.test_pick_word(pick_word)

Tests Passed


## Generating TV Script!!
This will generate the TV script for us.  Setting `gen_length` to the length of TV script we want to generate.

In [107]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'


loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: after this, uh, i ain't allowed to think i need no girls, huh.
lisa_simpson: you hate him?
rupert_murdoch: cut at the stagehand: i'm-so-stupid!
lisa_simpson: well-- i know, it's never all?
barney_gumble: work? spending springfield! do i had a little early who could do?
lenny_leonard: i just got person? i'm thinking?
homer_simpson: because he's just to ruin me!


kent_brockman: you are a woman actually cleaned up up beer.
homer_simpson:(to moe) y'know, now you're just gonna want to quit through...
homer_simpson: nobody actually know to me.


homer_simpson:(sympathetic) aw, you were gonna let you want to try up here.
lenny_leonard: yeah, i did, i've got a better friends?
lucius: hundred in the enemies world. i'm a pants. what you really like it.
moe_szyslak:(aside) not too time.. homer... your bike?...
homer_simpson: i know, i sure just down here


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  I trained on less than a megabyte of text.  In order to get good results, it's advised to use a smaller vocabulary or get more data. I didn't train on all the data, because that would take too long.  However, anyone is free to train the neural network on all the data.  